In [6]:
import os


In [7]:
%pwd

'd:\\Projects\\Fraud-Detection\\research'

In [8]:
%pwd
os.chdir("../")

In [28]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_n_estimators: int
    params_max_depth: int
    params_learning_rate: float
    params_objective: str
    params_booster: str
    params_random_state: int
    params_subsample: float
    params_colsample_bytree: float
    params_gamma: float
    params_min_child_weight: int
    params_scale_pos_weight: int
    params_reg_alpha: float
    params_reg_lambda: float


In [29]:

from fraud_detection.constants import *
from fraud_detection.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        xgb_params = self.params['xgboost_params']
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_n_estimators=xgb_params['n_estimators'],
            params_max_depth=xgb_params['max_depth'],
            params_learning_rate=xgb_params['learning_rate'],
            params_objective=xgb_params['objective'],
            params_booster=xgb_params['booster'],
            params_random_state=xgb_params['random_state'],
            params_subsample=xgb_params['subsample'],
            params_colsample_bytree=xgb_params['colsample_bytree'],
            params_gamma=xgb_params['gamma'],
            params_min_child_weight=xgb_params['min_child_weight'],
            params_scale_pos_weight=xgb_params['scale_pos_weight'],
            params_reg_alpha=xgb_params['reg_alpha'],
            params_reg_lambda=xgb_params['reg_lambda']
        )

        return prepare_base_model_config

In [31]:
import os
import urllib.request as request
from zipfile import ZipFile

In [32]:
import joblib
import xgboost as xgb

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        # Create XGBoost base model using all config parameters
        self.model = xgb.XGBClassifier(
            n_estimators=self.config.params_n_estimators,
            max_depth=self.config.params_max_depth,
            learning_rate=self.config.params_learning_rate,
            objective=self.config.params_objective,
            random_state=self.config.params_random_state,
            subsample=self.config.params_subsample,
            colsample_bytree=self.config.params_colsample_bytree,
            gamma=self.config.params_gamma,
            min_child_weight=self.config.params_min_child_weight,
            scale_pos_weight=self.config.params_scale_pos_weight,
            reg_alpha=self.config.params_reg_alpha,
            reg_lambda=self.config.params_reg_lambda,
            booster=self.config.params_booster
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, **kwargs):
        # For XGBoost, the base model is typically the full model
        return model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(model=self.model)
        joblib.dump(self.full_model, self.config.updated_base_model_path)

    @staticmethod
    def save_model(path: Path, model):
        joblib.dump(model, path)

In [33]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-08-30 12:32:42,661: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-30 12:32:42,666: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-30 12:32:42,668: INFO: common: created directory at: artifacts]
[2025-08-30 12:32:42,671: INFO: common: created directory at: artifacts/prepare_base_model]
